# Stability in graphs of knowledge

## Preliminaries

In [ ]:
import pandas as pd
#from bokeh.io import show, output_notebook
import geoviews as gv
#import cartopy.crs as ccrs
import geoviews.feature as gf
from scipy.spatial import Delaunay
import holoviews as hv
import numpy as np
#import networkx as nx
import random
from collections import Counter

hv.extension('bokeh')

## Find neighbours

In [ ]:
df = pd.read_excel('dfPoleis_v001.xlsx')
dfPoleis = df.dropna(subset=['latitude', 'longitude'], axis=0, inplace=False).reset_index(drop=True)
points=np.array(dfPoleis[["latitude", "longitude"]].values.tolist())

In [ ]:
triang = Delaunay(points)

def find_neighbors(pindex, triang):
    neighbors = []
    for simplex in triang.vertices:
        if pindex in simplex:
            neighbors.extend([simplex[i] for i in range(len(simplex)) if simplex[i] != pindex])
    return set(neighbors)

In [ ]:
valDict = {}
for value in ["latitude", "longitude"]:
    valDict[value] = [x for y in dfPoleis[[value]].astype(float).values.tolist() for x in y]
index = dfPoleis.index.values.tolist()
label = dfPoleis['city'].values.tolist()

Create links and start values for color distribution.

In [ ]:
yellowLength = round(2* len(index) /3)
blueLength = len(index) - yellowLength

colorList = ['yellow'] * yellowLength + ['blue'] * blueLength

In [ ]:
allLinks = []
Neighbors = []
for i in index:
    neigh = find_neighbors(i,triang)
    Neighbors.append((i,random.choice(colorList),neigh))
    for n in neigh:
        allLinks.append((i,n))

In [ ]:
print('Start values:',Counter([x[1] for x in Neighbors]))

In [ ]:
def mostCommon(lst):
    data = Counter(lst)
    most = data.most_common(2)
    try:
        fst_count = most[0][1]
    except:
        return ''
    try:
        scnd_count = most[1][1]
    except:
        return most[0][0]
    if fst_count > scnd_count:
        return most[0][0]
    elif fst_count == scnd_count:
        return ''

In [ ]:
def iterateColors(neighbourList):
    iterateColors = []
    for elem in neighbourList:
        ind = elem[0]
        color = elem[1]
        neigh = elem[2]
        if neigh:
            commonColor = mostCommon([neighbourList[n][1] for n in neigh])
            if commonColor:
                iterateColors.append((ind,commonColor,neigh))
            else:
                 iterateColors.append((ind,color,neigh))
        else:
             iterateColors.append((ind,color,neigh))     
    return iterateColors

In [ ]:
print('After iteration: ',Counter([x[1] for x in iterateColors(Neighbors)]))

## Model information flow

In [ ]:
N=100
startList = Neighbors
resultSet = {}
for i in range(N):
    iteration = iterateColors(startList)
    #if i % 10 == 0:
    resultSet[i] = [x[1] for x in iteration]
    #else:
    #    pass
    startList = iteration
    
print('After iteration: ',Counter([x[1] for x in iteration]))

All iterations of the simulation are stored in resultSet. This can be used for plotting the time-dependence.

## Merge city information with iteration result

In [ ]:
cities = dfPoleis[['city','region','latitude','longitude']].dropna(how='any').reset_index().rename(columns={'index':'node'})

allColorsFrame = pd.DataFrame(resultSet).stack().reset_index([0,1]).rename(columns={'level_0':'node','level_1':'iteration',0:'color'})

citiesIterated = pd.merge(cities,allColorsFrame,on='node')

## Generate nodes, graph and tiles

In [ ]:
routes = pd.DataFrame(allLinks,columns=['source','destination']).reset_index().rename(columns={'index':'node'})

In [ ]:
city_points_net = gv.Points(citiesIterated[citiesIterated.iteration == 10],['longitude','latitude'])

In [ ]:
nodes = gv.Nodes(city_points_net,['longitude','latitude','node'],['city','region','color'])

In [ ]:
graph = gv.Graph((routes, nodes), ['source', 'destination'], ['source', 'destination'])

In [ ]:
tiles = gv.WMTS('https://maps.wikimedia.org/osm-intl/{Z}/{X}/{Y}@2x.png')

In [ ]:
cdict = {'yellow':'yellow','blue':'blue'}

In [ ]:
city_points_data = gv.Dataset(citiesIterated, kdims=['city', 'region','iteration'])
    
city_points_iter = city_points_data.to(gv.Points, kdims=['longitude', 'latitude'],vdims=['city', 'region','color'])

## Display iterations on city level

In [ ]:
%%opts Overlay [width=1000 height=600] 
%%opts Points (size=5 cmap=cdict) [tools=['hover'] color_index='color']
%%opts Graph (edge_fill_alpha=0.3 edge_line_width=0.025)

tiles * city_points_iter

In [ ]:
# %%opts Graph [width=950 height=600] (edge_selection_line_color='black' edge_hover_line_color='red')
%%opts Graph (edge_line_width=1 edge_line_alpha=0.01 edge_nonselection_line_alpha=0.01)
%%opts Points (size=5 cmap=cdict) [tools=['hover'] color_index='color']
tiles * graph